In [2]:
import findspark
findspark.init() 

from pyspark.sql import SparkSession
from pyspark.sql.functions import mean as _mean, stddev as _stddev, count as _count, col, variance as _variance
from ETL.Extract import *
from ETL.Transform import *
import datetime

In [3]:
spark = SparkSession \
    .builder \
    .appName("Ep 2") \
    .getOrCreate()

In [4]:
df = readSampleDate(spark)
df = transformColumns(df)
df = transformData(df)


df.toPandas().columns

Index(['STATION', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'NAME', 'TEMP',
       'TEMP_ATTRIBUTES', 'DEWP', 'DEWP_ATTRIBUTES', 'SLP', 'SLP_ATTRIBUTES',
       'STP', 'STP_ATTRIBUTES', 'VISIB', 'VISIB_ATTRIBUTES', 'WDSP',
       'WDSP_ATTRIBUTES', 'MXSPD', 'GUST', 'MAX', 'MAX_ATTRIBUTES', 'MIN',
       'MIN_ATTRIBUTES', 'PRCP', 'PRCP_ATTRIBUTES', 'SNDP', 'FRSHTT'],
      dtype='object')

In [5]:
def mean(df, column_name, start_date, end_date):
    return df.filter((col("DATE") >= start_date) & \
                     (col("DATE") <= end_date)) \
             .select(_mean(column_name).alias(f'{column_name}_MEAN'))

In [6]:
def std(df, column_name, start_date, end_date):
    return df.filter((col("DATE") >= start_date) & \
                     (col("DATE") <= end_date)) \
             .select(_stddev(column_name).alias(f'{column_name}_STD'))

In [7]:
def variance(df, column_name, start_date, end_date):
    return df.filter((col("DATE") >= start_date) & \
                     (col("DATE") <= end_date)) \
             .select(_variance(column_name).alias(f'{column_name}_VARIANCE'))

In [28]:
def median(df, column_name, start_date, end_date):
    df = df.filter((col("DATE") >= start_date) & \
                     (col("DATE") <= end_date)) \
           .approxQuantile(column_name, [0.5],0.01)
    return df[0]

In [29]:
variance(df, "TEMP", datetime.datetime(2018, 1, 1), datetime.datetime(2018, 2, 1)).show()


+----------------+
|   TEMP_VARIANCE|
+----------------+
|24.0115111827957|
+----------------+



In [32]:
median(df, "TEMP", datetime.datetime(2018, 1, 1), datetime.datetime(2018, 2, 1))

1.5

In [34]:
print("Bem vindo! ")

Bem vindo! 
Bem vindo! 


In [35]:
import os

In [64]:
    end_date_str = input("Informe a data de término no formato dd/mm/yyyy: ")
    end_date = datetime.datetime.strptime(end_date_str, '%d/%m/%Y')

ValueError: time data '5/5/2005' does not match format '%d/%m/%Y: '

In [58]:
datetime.datetime.strptime("01/05/2020", '%d/%m/%Y')

datetime.datetime(2020, 5, 1, 0, 0)